In [4]:
# uncomment if needed

!pip install spacy preprocessor twint nltk
!python -m spacy download es_core_news_md


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4473 sha256=eb9c1efbfbcd5ff6fddca33bcf4f791f2ecdddefee63aca783906a894b498b29
  Stored in directory: /root/.cache/pip/wheels/b6/26/c1/ce3fdf93e63c11019da18dbeec4c09309d3edc5461c2a5fffb
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33912 sha256=72efb3c7894493894b79782961782f54822d1c889fb15ee2574288ba7040b0f2
  Stored in

In [5]:
import spacy
import preprocessor as p
import re
from nltk.corpus import stopwords
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [6]:
# Download Spanish stopwords from nltk
nltk.download('stopwords')

# Load Spanish language model in spaCy
nlp = spacy.load('es_core_news_md')

# Load Spanish stopwords
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [56]:
# add some custom stop words based on LDA results - keep adding
custom_stopwords = set([
    'ser', 'haber', 'hacer', 'tener', 'poder', 'ir', 'q', 'si', 'solo', 'saber', 'decir',
    'dar', 'querer', 'ver', 'así', 'sos', 'maje', 'dejar', 'si', 'solo', 'si', 'op', 'vos',
    'cada', 'mismo', 'usted', 'mas', 'pues', 'andar', 'ahora', 'claro', 'nunca', 'quedar', 'pasar',
    'venir', 'poner', 'dio', 'señora', 'señor', 'ahí', 'asi', 'vez', 'jajaja'
])
stop_words.update(custom_stopwords)

In [57]:
def preprocess_text(text):
    # Tokenize and lemmatize using spaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct and len(token.lemma_) > 2]
    return tokens

In [58]:
df = pd.read_csv('merged_file.csv')

In [59]:
# create new column for tokenized comments
df['cmt_processed_text'] = df['comment_processed_text'].apply(preprocess_text)

In [60]:
# Convert preprocessed text to document-term matrix
vectorizer = CountVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x, min_df=5)  # Custom tokenizer to use our preprocessed tokens
dtm = vectorizer.fit_transform(df['cmt_processed_text'])

# Get feature names (vocabulary)
terms = vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [61]:
# Initialize LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)  # Change n_components to the number of topics you want

# Fit the model to the document-term matrix
lda_model.fit(dtm)

# Get the topics
topics = lda_model.components_

In [62]:
def print_topics(model, terms, n_top_words=10):
    for idx, topic in enumerate(model.components_):
        print(f"Topic {idx+1}:")
        print([terms[i] for i in topic.argsort()[:-n_top_words - 1:-1]])

# Print the top words for each topic
print_topics(lda_model, terms)

Topic 1:
['diputado', 'marciano', 'gente', 'loco', 'viejo', 'pobre', 'salir', 'ganar', 'día', 'ladrón']
Topic 2:
['presidente', 'gracias', 'mejor', 'seguir', 'bukele', 'salvador', 'salvadoreño', 'país', 'año', 'nuevo']
Topic 3:
['pueblo', 'dios', 'salvadoreño', 'día', 'presidente', 'siempre', 'creer', 'deber', 'país', 'gracia']
Topic 4:
['pagar', 'deber', 'salir', 'ley', 'padre', 'tipo', 'llorar', 'hijo', 'verdad', 'bien']
Topic 5:
['trabajo', 'excelente', 'bien', 'ministro', 'buen', 'gobierno', 'esperar', 'hoy', 'bueno', 'deber']
